<h3  style = "color: blue" >Importing Libraries<h3>

In [7]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


<h3 style = "color: blue" >GPU Acceleration</h3>

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

<h3  style = "color: blue" >Data Ingestion and Transformation<h3>

In [9]:
dataset_path = "../dataset"

#mean and std values are just the average color and color spread of millions of ImageNet images.
image_transforms = transforms.Compose([
    
    #Data Augumentaion
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast= 0.2),

    #Normal Trnasformation
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])    
    
])

dataset = datasets.ImageFolder(root= dataset_path, transform= image_transforms)

<h3 style = "color:blue" >Dataset Overview</h3>

In [10]:

print(f"The Size of Dataset: {len(dataset)}")
print(f"The number of classes: {len(dataset.classes)}")
print("==" * 15)
print("\nThe Classes:")
print(dataset.classes)

The Size of Dataset: 2300
The number of classes: 6

The Classes:
['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']


<h3 style = "color:blue" >Splitting dataset to test and train</h3>

In [11]:
train_size = int(len(dataset) * 0.75)
val_size = int(len(dataset) - train_size)

train_dataset, val_dataset = random_split(dataset= dataset, lengths=[train_size, val_size])

print("After Random Splitting:\n")
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")


After Random Splitting:

Train size: 1725
Validation size: 575


<h3 style = "color:blue" >Dividing into Batches</h3>

In [12]:
batch_size = 32

train_loader = DataLoader(dataset= train_dataset, batch_size= batch_size, shuffle= True)
val_loader = DataLoader(dataset= val_dataset, batch_size= batch_size, shuffle= True)

<h3 style = "color:blue" >Model Training & Hyperparameter Tuning</h3>

In [13]:
# Load the pre-trained ResNet model
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True            
            
        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [14]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    
    # Load the model
    model = CarClassifierResNet(num_classes= 6, dropout_rate=dropout_rate).to(device)
    
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    # Training loop (using fewer epochs for faster hyperparameter tuning)
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.inference_mode():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        # Report intermediate result to Optuna
        trial.report(accuracy, epoch)
        
        # Handle pruning (if applicable)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time: {end - start} seconds")
    
    return accuracy

In [ ]:
# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

In [16]:
study.best_params

{'lr': 0.0005215133558173996, 'dropout_rate': 0.31909112035069076}